In [1]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes
#!pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 69.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 91.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.4/418.4 kB 11.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.9 MB/s eta 0:00:0000:0100:01


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, MistralForCausalLM
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model #prepare_model_for_int8_training
from datasets import load_dataset
import transformers

from huggingface_hub import hf_hub_download

2024-05-29 16:09:16.235854: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 16:09:16.235967: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 16:09:16.393454: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
transformers.__version__

In [ ]:
model_name = "LoneStriker/BioMistral-7B-SLERP-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main")


tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [5]:
model.eval() # model in evaluation mode (dropout modules are deactivated)

# craft prompt
comment = "trouble digestif?"
prompt=f'''[INST] {comment} [/INST]'''

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=50)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] trouble digestif? [/INST] If you are experiencing digestive issues, it is important to consult with a healthcare professional to determine the underlying cause. There are many potential causes of digestive problems, including infections, food allergies, irritable bowel syndrome, and more.


In [21]:
intstructions_string = f"""You are an expert in medical knowledge. Please answer the user's question with a paraphrase,\
explanation or short definition. If you don't know the answer, just say that you don't know, don't try to make up an answer.\
Only return the helpful answer. 

Answer must be clear, concise and easy to understand for lay people. \

Term  :"""

inst_string = f"""Vous êtes un expert en médecine. Utilisez les informations
suivantes pour répondre à la question de l'utilisateur par une paraphrase,
une explication ou une courte définition.
Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas,
n'essayez pas d'inventer une réponse.
Ne renvoyez que la réponse utile. La réponse doit être claire, concise et facile
à comprendre pour le grand public.

Term : """

prompt_template = lambda comment: f'''[INST] {inst_string} {comment} \n[/INST]'''

prompt = prompt_template(comment)

prompt = """[INST] Expliquez-moi le terme médical en mots simples, par une paraphrase ou une courte définition :  causes courantes de syndrome confusionnel [/INST]"""

print(prompt)

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=40)
print('#'*50)
print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Expliquez-moi le terme médical en mots simples, par une paraphrase ou une courte définition :  causes courantes de syndrome confusionnel [/INST]
##################################################
<s> [INST] Expliquez-moi le terme médical en mots simples, par une paraphrase ou une courte définition :  causes courantes de syndrome confusionnel [/INST] Le syndrome confusionnel est une maladie médicale qui se traduit par une perte de conscience ou une confusion. Les causes courantes de ce syndrome sont les infections, les bless


## Prepare dataset

In [35]:
intstructions_string = f"""Vous êtes un expert en médecine. Utilisez les informations suivantes pour répondre à la question de l'utilisateur par une paraphrase, une explication ou une courte définition. Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse. Ne renvoyez que la réponse utile. La réponse doit être claire, concise et facile à comprendre pour le grand public. Term :  """
#Helpful answer: """
intstructions_string = f"""Expliquez-moi le terme médical en mots simples, par une paraphrase ou une courte définition : """
example_template = lambda term, paraphrase: f'''<s>[INST] {intstructions_string} {term} [/INST] {paraphrase} </s>'''

import pandas as pd
valdf = pd.read_csv('/kaggle/input/refomed/refomed_val.csv', sep='\t')
testdf = pd.read_csv('/kaggle/input/refomed/refomed_test.csv', sep='\t')
traindf= pd.read_csv('/kaggle/input/refomed/refomed_train.csv', sep='\t')

from datasets import Dataset, DatasetDict

train_example_list = []
val_example_list = []
test_example_list = []
for i in range(len(traindf.term.values)):
    example = example_template(traindf.term.values[i],traindf.paraphrase.values[i])
    train_example_list.append(example)
    
for i in range(len(valdf.term.values)):
    example = example_template(valdf.term.values[i],valdf.paraphrase.values[i])
    val_example_list.append(example)
    
for i in range(len(testdf.term.values)):
    example = example_template(testdf.term.values[i],testdf.paraphrase.values[i])
    test_example_list.append(example)
    
trds = Dataset.from_pandas(pd.DataFrame({"example": train_example_list[:50]}))
vds = Dataset.from_pandas(pd.DataFrame({"example": val_example_list[:50]}))
tsds = Dataset.from_pandas(pd.DataFrame({"example": test_example_list}))

ds = DatasetDict()

ds['train'] = trds
ds['dev'] = vds
ds['test'] = tsds

data = ds
data

DatasetDict({
    train: Dataset({
        features: ['example'],
        num_rows: 50
    })
    dev: Dataset({
        features: ['example'],
        num_rows: 50
    })
    test: Dataset({
        features: ['example'],
        num_rows: 1253
    })
})

In [36]:
# load dataset
#data = load_dataset("shawhin/shawgpt-youtube-comments")
data

# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["example"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="pt",
        #truncation=True,
        padding='max_length',
        max_length=512
    )

    return tokenized_inputs

# tokenize training and validation datasets
tokenized_data = data.map(tokenize_function, batched=True)

# setting pad token
tokenizer.pad_token = tokenizer.eos_token
# data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/1253 [00:00<?, ? examples/s]

## Tuning the model

In [37]:
print(tokenized_data['dev'][0])

{'example': '<s>[INST] Expliquez-moi le terme médical en mots simples, par une paraphrase ou une courte définition :  causes courantes de syndrome confusionnel [/INST] trouble métabolique , méningite , crises convulsives répétées , accident vasculaire , etc </s>', 'input_ids': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

In [38]:
model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 2,097,152 || all params: 264,507,392 || trainable%: 0.7929


In [41]:
# hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 3

In [40]:
!pip install huggingface_hub


!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('<hftoken>')"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [42]:
# define training arguments

training_args = transformers.TrainingArguments(
    output_dir= "biomistral-gptq-ft",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to = "none",
    push_to_hub = True

)

# configure trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["dev"],
    args=training_args,
    data_collator=data_collator
)

# train model

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,4.593500,4.051593
1,4.105100,3.609291
2,2.895500,3.408180


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=9, training_loss=3.8646971384684243, metrics={'train_runtime': 358.969, 'train_samples_per_second': 0.418, 'train_steps_per_second': 0.025, 'total_flos': 57387893391360.0, 'train_loss': 3.8646971384684243, 'epoch': 2.77})

In [ ]:
from huggingface_hub import Repository

# Define the repository where you want to push your model
repo_name = "amasi/biomistral-gptq-ft"
#!cd /kaggle/working/ && git init && git remote add origin && git pull origin main
# Create a repository object
repo = Repository(local_dir="/kaggle/working/", clone_from=repo_name)

# Push the trained model to the repository
trainer.save_model("fine-tuned-model")
repo.push_to_hub(commit_message="Initial model upload")

## Dataset conversion txt to dataset instance

In [ ]:
# renable warnings
model.config.use_cache = True

intstructions_string = f"""ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""
prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

comment = "Great content, thank you!"

prompt = prompt_template(comment)
print(prompt)

In [ ]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

comment = "What is fat-tailedness?"
prompt = prompt_template(comment)

model.eval()
inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)
print(tokenizer.batch_decode(outputs)[0])